# Data Preprocessing

## 1. Initial data loading

Import necessary libraries and load various datasets into dataframes.

In [150]:
import pandas as pd
import numpy as np

chapter_df = pd.read_csv('../data/chapter-data.csv')
listing_session_df = pd.read_csv('../data/listing-sessions-2018-2020.csv')
participant_df = pd.read_csv('../data/participant-data.csv')
program_location_df = pd.read_csv('../data/program-locations.csv')
session_registration_df = pd.read_csv('../data/session-registration.csv')

/var/folders/xy/m0gg_ygd339crs0vw79bl00h0000gn/T/ipykernel_6971/4204468503.py:5: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  listing_session_df = pd.read_csv('../data/listing-sessions-2018-2020.csv')


## 2. Merge datasets

Merge all datasets to create comprehensive dataset. This step is crucial for subsequent analysis.

In [151]:
# Merge participant and session registration datasets
merged_df = pd.merge(participant_df, session_registration_df, how='outer', on='contact_id')

# Merge new dataset with listing session dataset
merged_df = pd.merge(merged_df, listing_session_df,  how='outer', on=['listing_session_id', 'chapter_name'])

# Merge new dataset with chapter dataset
merged_df = pd.merge(merged_df, chapter_df,  how='outer', on=['chapter_name', 'chapter_id'])

# Merge new dataset with program location dataset
merged_df = pd.merge(merged_df, program_location_df,  how='outer', on=['chapter_name', 'billing_state', 
                                                                       'program_name', 'program_id', 'billing_city', 
                                                                       'chapter_id', 'billing_zipcode', 'billing_street'])

## 3. Data cleaning

Remove columns that are not required for analysis to simplify dataset.

In [152]:
df = merged_df.drop(columns=['session_registration_id',
       'chapter_name', 'program_name',
       'register_start_datetime', 'register_end_datetime',
       'session_start_datetime', 'session_end_datetime', 'billing_street',
       'billing_city', 'billing_state', 'billing_zipcode', 'billing_country'])

df.head()

,contact_id,age,gender,ethnicity,grade_level,mailing_zipcode,participation_year,session_registration_num,listing_session_id,session_registration_status,...,waitlist_capacity,waitlist_space,total_registrations,chapter_active,governance_structure,chapter_territory,market_size,chapter_region,youth_population,program_active
0,FK-000001,11.0,Male,White or Caucasian,NaN,29803,2022.0,SR-1132745,a0H1R000014upfLUAQ,Registered,...,NaN,0.0,105.0,Yes,Single-Purpose | Facility-Operated,Eastern,Smaller Population,Southeast,16781.0,NaN
1,FK-000001,14.0,Male,White or Caucasian,NaN,29803,2021.0,SR-1132745,a0H1R000014upfLUAQ,Registered,...,NaN,0.0,105.0,Yes,Single-Purpose | Facility-Operated,Eastern,Smaller Population,Southeast,16781.0,NaN
2,FK-000002,5.0,Female,White or Caucasian,NaN,29803,2022.0,SR-1132845,a0H1R000014upfLUAQ,Registered,...,NaN,0.0,105.0,Yes,Single-Purpose | Facility-Operated,Eastern,Smaller Population,Southeast,16781.0,NaN
3,FK-000003,11.0,Male,White or Caucasian,2,29803,2022.0,SR-1132740,a0H1R000014upfLUAQ,Registered,...,NaN,0.0,105.0,Yes,Single-Purpose | Facility-Operated,Eastern,Smaller Population,Southeast,16781.0,NaN
4,FK-000003,5.0,Male,White or Caucasian,2,29803,2021.0,SR-1132740,a0H1R000014upfLUAQ,Registered,...,NaN,0.0,105.0,Yes,Single-Purpose | Facility-Operated,Eastern,Smaller Population,Southeast,16781.0,NaN


## 4. Aggregation of variables

Create aggregated features like the first and last year of participation, demographics, etc.

In [153]:
# Create features
aggregate_df = df.groupby('contact_id').agg(
    first_participation_year = pd.NamedAgg(column='participation_year', aggfunc='min'),
    last_participation_year = pd.NamedAgg(column='participation_year', aggfunc='max'),
    age_at_first_participation = pd.NamedAgg(column='age', aggfunc='first'),
    age_at_last_participation = pd.NamedAgg(column='age', aggfunc='last'),
    gender = pd.NamedAgg(column='gender', aggfunc='first'),
    ethnicity = pd.NamedAgg(column='ethnicity', aggfunc='first')
).reset_index()

# Calculate age at last participation
aggregate_df['age_at_last_participation'] = aggregate_df['age_at_first_participation'] + (aggregate_df['last_participation_year'] - aggregate_df['first_participation_year'])

aggregate_df.head()

,contact_id,first_participation_year,last_participation_year,age_at_first_participation,age_at_last_participation,gender,ethnicity
0,FK-000001,2021.0,2022.0,11.0,12.0,Male,White or Caucasian
1,FK-000002,2022.0,2022.0,5.0,5.0,Female,White or Caucasian
2,FK-000003,2018.0,2022.0,11.0,15.0,Male,White or Caucasian
3,FK-000004,2018.0,2022.0,15.0,19.0,Male,White or Caucasian
4,FK-000005,2018.0,2022.0,6.0,10.0,Female,Multi-Racial


## 5. Calculating unique counts and rates

Calculate unique counts and rates for different entities like chapters and programs to understand participant engagement.

### Calculating unique chapter counts

Calculate the number of unique chapters each participant is associated with. This provides insights into the diversity of chapters a participant is engaged with, which can be an important factor in participation retention.

In [154]:
unique_chapters = df.groupby('contact_id')['chapter_id'].nunique().reset_index()
unique_chapters.columns = ['contact_id', 'unique_chapters_count']
unique_chapters = unique_chapters.sort_values(by='unique_chapters_count', ascending=False).reset_index(drop=True)
unique_chapters['multiple_chapters'] = (unique_chapters['unique_chapters_count'] > 1).astype(int)

unique_chapters.head()

,contact_id,unique_chapters_count,multiple_chapters
0,FK-124452,2,1
1,FK-072860,2,1
2,FK-167064,2,1
3,FK-045181,2,1
4,FK-113123,2,1


### Calculating unique program counts

Determine the number of unique programs each participant is involved in. A higher count could indicate a broader engagement with the organization's offerings.

In [155]:
unique_programs = df.groupby('contact_id')['program_id'].nunique().reset_index()
unique_programs = unique_programs[unique_programs['program_id'] != 0]
unique_programs.columns = ['contact_id', 'unique_programs_count']
unique_programs['multiple_programs'] = (unique_programs['unique_programs_count'] > 1).astype(int)

# Merge unique chapter and unique program datasets
chapters_programs_df = pd.merge(unique_chapters, unique_programs, how='outer', on='contact_id')

chapters_programs_df.head()

,contact_id,unique_chapters_count,multiple_chapters,unique_programs_count,multiple_programs
0,FK-124452,2,1,1.0,0.0
1,FK-072860,2,1,2.0,1.0
2,FK-167064,2,1,2.0,1.0
3,FK-045181,2,1,3.0,1.0
4,FK-113123,2,1,3.0,1.0


### Calculating session participation rates

Calculate the rate of session participation for each participant. This rate is a ratio of the number of sessions a participant attended to the total number of sessions offered, providing a metric for participant engagement in these sessions.

In [156]:
# Calculate number of unique sessions
unique_sessions = df.groupby(['contact_id', 'chapter_id', 'participation_year'])['session_registration_num'].nunique().reset_index()
unique_sessions.columns = ['contact_id', 'chapter_id', 'participation_year', 'unique_sessions_count']

# Calculate number of total sessions
total_sessions = df.groupby(['chapter_id', 'participation_year'])['listing_session_id'].nunique().reset_index()
total_sessions.columns = ['chapter_id', 'participation_year', 'total_programs_count']

# Merge unique and total session datasets
sessions_df = pd.merge(unique_sessions, total_sessions, how='outer', on=['chapter_id', 'participation_year'])

# Fill all NA values with 0
sessions_df['unique_sessions_count'] = sessions_df['unique_sessions_count'].fillna(0)
sessions_df['total_programs_count'] = sessions_df['total_programs_count'].fillna(0)

# Calculate session rate
sessions_df['sessions_rate'] = sessions_df['unique_sessions_count'] / sessions_df['total_programs_count']

# Calculate average session rate per participant
avg_sessions_df = sessions_df.groupby(['contact_id'])['sessions_rate'].mean().reset_index()

avg_sessions_df.head()

,contact_id,sessions_rate
0,FK-000001,0.011431
1,FK-000002,0.006250
2,FK-000003,0.017147
3,FK-000004,0.017147
4,FK-000005,0.022296


### Calculating private event rate

Calculate private event rate, which is the ratio of private events offered over how many the participant attended. This could provide information on if private events are helping or hurting participation retention.

In [157]:
# Calculate number of unique private events per participant
unique_private = df.groupby(['contact_id', 'chapter_id', 'participation_year'])['private_event'].sum().reset_index()
unique_private.columns = ['contact_id', 'chapter_id', 'participation_year', 'participant_private']

# Calculate number of total private events per chapter
unique_private_total = df.groupby(['chapter_id', 'participation_year'])['private_event'].sum().reset_index()
unique_private_total.columns = ['chapter_id', 'participation_year', 'total_private']

# Merge unique and total private session datasets
private_df = pd.merge(unique_private, unique_private_total, how='outer', on=['chapter_id', 'participation_year'])

# Fill all NA values with 0
private_df['participant_private'] = private_df['participant_private'].fillna(0)
private_df['total_private'] = private_df['total_private'].fillna(0)

# Calculate average private event rate
private_df['avg_private'] = private_df['participant_private'] / private_df['total_private']

# Calculate average private event rate per participant
private_participant_df = private_df.groupby(['contact_id'])['avg_private'].mean().reset_index()

private_participant_df.head()

,contact_id,avg_private
0,FK-000001,NaN
1,FK-000002,NaN
2,FK-000003,NaN
3,FK-000004,NaN
4,FK-000005,NaN


### Calculating session price rates

Calculate average session price that the participant attended. Higher prices may influence participation retention.

In [158]:
# Calculate average session price per participant
price = df.groupby(['contact_id', 'chapter_id', 'participation_year'])['base_price'].mean().reset_index()
price.columns = ['contact_id', 'chapter_id', 'participation_year', 'participant_price']

# Calculate average session price per chapter
total_price = df.groupby(['chapter_id', 'participation_year'])['base_price'].mean().reset_index()
total_price.columns = ['chapter_id', 'participation_year', 'total_base']

# Merge participant average and chapter average datasets
price_df = pd.merge(price, total_price, how='outer', on=['chapter_id', 'participation_year'])

# Calculate average price rate
price_df['avg_price'] = price_df[['participant_price', 'total_base']].mean(axis=1)

# Calculate average price rate per participant
price_participant = price_df.groupby(['contact_id'])['avg_price'].mean().reset_index()

### Calculate the distribution of program types attended

Calculate program type distribution. This tells us what types of programs were offered and which ones participants attended.

In [159]:
# Calculate number of unique program types participant attended
program_type = df.groupby(['contact_id', 'chapter_id', 'participation_year'])['program_type'].nunique().reset_index()

# Calculate total number of sessions per program type
program_group = df.groupby(['contact_id', 'participation_year', 'program_type'])['session_registration_num'].count().reset_index()

# Create pivot table representing participant program type participation distribution
pivot_table = program_group.pivot_table(index=['contact_id', 'participation_year'],
                                        columns='program_type',
                                        values='session_registration_num',
                                        fill_value=0).reset_index()
pivot_table.columns = ['contact_id', 'participation_year', 'camps_sessions', 'general_programs_sessions']

# Calculate total number of sessions
pivot_table['total'] = pivot_table['camps_sessions'] + pivot_table['general_programs_sessions']

# Calculate camp type average
pivot_table['camp_rate'] = pivot_table['camps_sessions'] / pivot_table['total']

# Calculate general program type average
pivot_table['general_programs_rate'] = pivot_table['general_programs_sessions'] / pivot_table['total']

# Calculate camp type rate
camp_participant = pivot_table.groupby(['contact_id'])['camp_rate'].mean().reset_index()

# Calculate general program type rate
general_participant = pivot_table.groupby(['contact_id'])['general_programs_rate'].mean().reset_index()

# Merge camp type rate and general program type rate datasets
program_rates = pd.merge(general_participant, camp_participant, how='outer', on='contact_id')

program_rates.head()

,contact_id,general_programs_rate,camp_rate
0,FK-006785,1.0,0.0
1,FK-006806,1.0,0.0
2,FK-006833,1.0,0.0
3,FK-006837,1.0,0.0
4,FK-006839,1.0,0.0


### Calcaulate parent communication rate

Calculate the sessions (per chapter) that offered parent communication. A lack of parent communication could reduce the likelihood of the child retaining in the program.

In [160]:
# Filter out NA values
parent_df = df[df['parent_communication'].notnull()]

# Calculate number of parent communication values that are null and not null
parent_counts = df.groupby(['contact_id', 'chapter_id', 'participation_year']).agg(
    parent_communication_not_null = pd.NamedAgg(column='parent_communication', aggfunc=lambda x: x.notnull().sum()),
    parent_communication_null = pd.NamedAgg(column='parent_communication', aggfunc=lambda x: x.isnull().sum())
).reset_index()

# Create binary value if parent communication is present or not
parent_counts['parent_not'] = parent_counts['parent_communication_not_null'].apply(lambda x: 1 if x > 0 else 0)
parent_counts['parent_null'] = parent_counts['parent_communication_null'].apply(lambda x: 1 if x > 0 else 0)

# Calculate total number of sessions where parent communication is present vs not present
participant_not_null = parent_counts.groupby(['contact_id', 'chapter_id', 'participation_year'])['parent_not'].sum().reset_index()
participant_null = parent_counts.groupby(['contact_id', 'chapter_id', 'participation_year'])['parent_null'].sum().reset_index()

# Merge null and not null parent communication datasets
participant_parent_df = pd.merge(participant_not_null, participant_null, how='outer', on=['contact_id', 'chapter_id', 'participation_year'])

# Calculate parent communication not present rate
participant_parent_df['parent_comm_total'] = participant_parent_df['parent_not'] + participant_parent_df['parent_null']
participant_parent_df['parent_participant_rate'] = participant_parent_df['parent_not'] / participant_parent_df['parent_comm_total']

# Calculate average parent communication not present rate (per participant)
participant_parent_df = participant_parent_df.groupby(['contact_id'])['parent_participant_rate'].mean().reset_index()


# Calculate null and not null parent communication per chapter
chapter_counts = df.groupby(['chapter_id', 'participation_year', 'listing_session_name']).agg(
    parent_communication_not_null = pd.NamedAgg(column='parent_communication', aggfunc=lambda x: x.notnull().sum()),
    parent_communication_null = pd.NamedAgg(column='parent_communication', aggfunc=lambda x: x.isnull().sum())
).reset_index()

# Create binary value if parent communication is present or not
chapter_counts['parent_not'] = chapter_counts['parent_communication_not_null'].apply(lambda x: 1 if x > 0 else 0)
chapter_counts['parent_null'] = chapter_counts['parent_communication_null'].apply(lambda x: 1 if x > 0 else 0)

# Calculate total number of sessions where parent communication is present vs not present (per chapter)
chapter_not_null = chapter_counts.groupby(['chapter_id', 'participation_year'])['parent_not'].sum().reset_index()
chapter_null = chapter_counts.groupby(['chapter_id', 'participation_year'])['parent_null'].sum().reset_index()

# Merge null and not null parent communication datasets (per chapter)
chapter_parent_df = pd.merge(chapter_not_null, chapter_null, how='outer', on=['chapter_id', 'participation_year'])

# Calculate parent communication not present rate
chapter_parent_df['parent_comm_total'] = chapter_parent_df['parent_not'] + chapter_parent_df['parent_null']
chapter_parent_df['parent_chapter_rate'] = chapter_parent_df['parent_not'] / chapter_parent_df['parent_comm_total']

# Calculate average parent communication not present rate (per chapter)
chapter_parent_df = chapter_parent_df.groupby(['chapter_id'])['parent_chapter_rate'].mean().reset_index()

# Add contact id to chapter dataset to merge with large dataset
chapter_participant_df = df[['contact_id', 'chapter_id']].dropna()
chapter_participant_df = chapter_participant_df.drop_duplicates()
chapter_parent_df = pd.merge(chapter_parent_df, chapter_participant_df, how='outer', on='chapter_id')

### Calculate session age range and residual

Calculate the average age range for each session each participant attended. Then calculate a residual of the average age range and participant age. If sessions for younger kids are only offered, older participants may be less likely to participate.

In [161]:
# Create simplified dataset for age range
age_range_df = df[['contact_id', 'age', 'chapter_id', 'program_id', 'participation_year', 'listing_session_name',
               'min_age', 'max_age']]

# Calculate average age range per session
age_range_df['age_avg'] = (age_range_df['min_age'] + age_range_df['max_age']) / 2

# Calculate residual per session
age_range_df['residual'] = age_range_df['age_avg'] - age_range_df['age']

# Calculate average residual per participant per participation year
residual_avg_df = age_range_df.groupby(['contact_id', 'participation_year'])['residual'].mean().reset_index()

# Calculate average residual per participant overall
residual_avg_df = age_range_df.groupby(['contact_id'])['residual'].mean().reset_index()

residual_avg_df.head()

/var/folders/xy/m0gg_ygd339crs0vw79bl00h0000gn/T/ipykernel_6971/1461390606.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_range_df['age_avg'] = (age_range_df['min_age'] + age_range_df['max_age']) / 2
/var/folders/xy/m0gg_ygd339crs0vw79bl00h0000gn/T/ipykernel_6971/1461390606.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_range_df['residual'] = age_range_df['age_avg'] - age_range_df['age']


,contact_id,residual
0,FK-000001,-0.500000
1,FK-000002,6.500000
2,FK-000003,2.500000
3,FK-000004,-2.833333
4,FK-000005,2.500000


### Calculate session length

Calculate the length (in days) of each session and the average length of sessions the participant attended. Longer or shorter time periods could affect participant retention, especially with participants that do not have a lot of time in their schedule.

In [162]:
# Convert start and end dates to datetime
df['session_start_date'] = pd.to_datetime(df['session_start_date'], format='%m/%d/%y')
df['session_end_date'] = pd.to_datetime(df['session_end_date'], format='%m/%d/%y')

# Calculate duration in days
df['duration'] = (df['session_end_date'] - df['session_start_date']).dt.days

# Calculate average duration per participant
duration_participant = df.groupby(['contact_id', 'chapter_id'])['duration'].mean().reset_index(name='avg_duration')

# Calculate average duration per chapter
duration_chapter = df.groupby(['chapter_id'])['duration'].mean().reset_index(name='avg_duration_chapter')

# Merge average participant duration and chapter duration datasets
duration_df = pd.merge(duration_participant, duration_chapter, how='outer', on=['chapter_id'])

### Merge all counts and rates into main dataset

In [163]:
# Merge aggregate dataset with chapter and program rate dataset and 
aggregate_df = pd.merge(aggregate_df, chapters_programs_df, how='outer', on='contact_id')

# Merge new dataset with average session rate dataset
aggregate_df = pd.merge(aggregate_df, avg_sessions_df, how='outer', on='contact_id')

# Merge new dataset with average private event dataset
aggregate_df = pd.merge(aggregate_df, private_participant_df, how='outer', on='contact_id')

# Merge new dataset with average session price dataset
aggregate_df = pd.merge(aggregate_df, price_participant, how='outer', on='contact_id')

# Merge new dataset with average program type rate dataset
aggregate_df = pd.merge(aggregate_df, program_rates, how='outer', on='contact_id')

# Merge new dataset with parent communication (per participant) dataset
aggregate_df = pd.merge(aggregate_df, participant_parent_df, how='outer', on='contact_id')

# Merge new dataset with parent communication (per chapter) dataset
aggregate_df = pd.merge(aggregate_df, chapter_parent_df, how='outer', on='contact_id')

# Merge new dataset with average age range residual dataset
aggregate_df = pd.merge(aggregate_df, residual_avg_df, how='outer', on='contact_id')

# Merge new dataset with average session duration dataset
aggregate_df = pd.merge(aggregate_df, duration_df, how='outer', on=['contact_id', 'chapter_id'])

## 6. Data transformation and feature engineering

Calculate participation length (in years) so that we can drop the variables that directly lead to data leakage.

In [164]:
aggregate_df['participation_length'] = aggregate_df['last_participation_year'] - aggregate_df['first_participation_year']
aggregate_df = aggregate_df.dropna(subset='contact_id')

aggregate_df.head()

,contact_id,first_participation_year,last_participation_year,age_at_first_participation,age_at_last_participation,gender,ethnicity,unique_chapters_count,multiple_chapters,unique_programs_count,...,avg_price,general_programs_rate,camp_rate,parent_participant_rate,chapter_id,parent_chapter_rate,residual,avg_duration,avg_duration_chapter,participation_length
0,FK-000001,2021.0,2022.0,11.0,12.0,Male,White or Caucasian,1.0,0.0,1.0,...,27.369535,NaN,NaN,0.0,0013600000s0Su5AAE,0.078019,-0.500000,NaN,NaN,1.0
1,FK-000002,2022.0,2022.0,5.0,5.0,Female,White or Caucasian,1.0,0.0,1.0,...,26.130410,NaN,NaN,0.0,0013600000s0Su5AAE,0.078019,6.500000,NaN,NaN,0.0
2,FK-000003,2018.0,2022.0,11.0,15.0,Male,White or Caucasian,1.0,0.0,1.0,...,37.369535,NaN,NaN,0.0,0013600000s0Su5AAE,0.078019,2.500000,NaN,NaN,4.0
3,FK-000004,2018.0,2022.0,15.0,19.0,Male,White or Caucasian,1.0,0.0,1.0,...,27.369535,NaN,NaN,0.0,0013600000s0Su5AAE,0.078019,-2.833333,NaN,NaN,4.0
4,FK-000005,2018.0,2022.0,6.0,10.0,Female,Multi-Racial,1.0,0.0,1.0,...,51.417931,NaN,NaN,0.5,0013600000s0Su5AAE,0.078019,2.500000,NaN,NaN,4.0


## 7. Creation of target variable and change column types

Define the target variable for the model, indicating whether a participant retained in the program. Convert first and last participation year and first and last ages to integers.

In [165]:
aggregate_df['retained'] = aggregate_df['last_participation_year'].apply(lambda x: 1 if x == 2022 else 0)
aggregate_df['first_participation_year'] = aggregate_df['first_participation_year'].astype(int)
aggregate_df['last_participation_year'] = aggregate_df['last_participation_year'].astype(int)
aggregate_df['age_at_first_participation'] = aggregate_df['age_at_first_participation'].astype(int)
aggregate_df['age_at_last_participation'] = aggregate_df['age_at_last_participation'].astype(int)

aggregate_df.head()

,contact_id,first_participation_year,last_participation_year,age_at_first_participation,age_at_last_participation,gender,ethnicity,unique_chapters_count,multiple_chapters,unique_programs_count,...,general_programs_rate,camp_rate,parent_participant_rate,chapter_id,parent_chapter_rate,residual,avg_duration,avg_duration_chapter,participation_length,retained
0,FK-000001,2021,2022,11,12,Male,White or Caucasian,1.0,0.0,1.0,...,NaN,NaN,0.0,0013600000s0Su5AAE,0.078019,-0.500000,NaN,NaN,1.0,1
1,FK-000002,2022,2022,5,5,Female,White or Caucasian,1.0,0.0,1.0,...,NaN,NaN,0.0,0013600000s0Su5AAE,0.078019,6.500000,NaN,NaN,0.0,1
2,FK-000003,2018,2022,11,15,Male,White or Caucasian,1.0,0.0,1.0,...,NaN,NaN,0.0,0013600000s0Su5AAE,0.078019,2.500000,NaN,NaN,4.0,1
3,FK-000004,2018,2022,15,19,Male,White or Caucasian,1.0,0.0,1.0,...,NaN,NaN,0.0,0013600000s0Su5AAE,0.078019,-2.833333,NaN,NaN,4.0,1
4,FK-000005,2018,2022,6,10,Female,Multi-Racial,1.0,0.0,1.0,...,NaN,NaN,0.5,0013600000s0Su5AAE,0.078019,2.500000,NaN,NaN,4.0,1


## 8. Preparing data for exploratory data analysis (EDA)

Export the preprocessed data for further exploratory data analysis.

In [166]:
aggregate_df = aggregate_df.dropna(subset=['contact_id'])
aggregate_df.to_csv('../data/data-preprocessed-EDA.csv', index=False)

## 9. Preprocessing for modeling

Encode categorical variables and standardize numerical features, preparing the data for modeling.

In [174]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Encode categorical variables
df_encoded = pd.get_dummies(aggregate_df, columns=['gender', 'ethnicity'])

# Drop columns
df_features = df_encoded.drop(columns=['contact_id', 'chapter_id', 'first_participation_year', 'last_participation_year', 'avg_private', 'general_programs_rate', 'camp_rate', 'retained'])

# Impute variables
num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')

for column in df_features.columns:
    if df_features[column].dtype == 'float64':
        df_features[column] = num_imputer.fit_transform(df_features[[column]])
    elif df_features[column].dtype == 'bool':
        df_features[column] = df_features[column].astype('int')
        df_features[column] = cat_imputer.fit_transform(df_features[[column]])
        df_features[column] = df_features[column].replace({'missing': None}).astype('bool')

# Scale numerical variables
numerical_cols = df_features.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
df_features[numerical_cols] = scaler.fit_transform(df_features[numerical_cols])
df_features['retained'] = df_encoded['retained']

## 10. Final data export for modeling

The final step where the preprocessed dataset is exported for use in model training.

In [176]:
df_features.to_csv('../data/data-preprocessed-modeling.csv', index=False)